In [4]:
from sklearn.datasets import make_blobs
import pandas as pd
import numpy as np
dataset, _ = make_blobs(n_samples=1000, n_features=4, centers=3, cluster_std=0.6, random_state=0)
dataset = pd.DataFrame(dataset, columns = ['x1', 'x2', 'x3', 'x4'])

In [5]:
dataset.head(2)

,x1,x2,x3,x4
0,-1.332890,2.096248,-1.708052,8.604367
1,1.275484,4.932571,2.225835,1.943265


클러스터링 평가 지표에는 Dunn Index와 Silhouette Score가 있다. 

Dunn Index와 Silhouette Score를 직접 구해보자.

### 1. KMeans를 이용해서 군집화를 진행하시오. 
- 파라미터 설정
    - n_clusters = 3 
    - random_state = 0
    - 나머지는 default 값을 따른다
- sklearn.cluster.KMeans 사용

In [6]:
from sklearn.cluster import KMeans

# 군집화 수행
kmeans = KMeans(n_clusters=3, random_state=0).fit(dataset)

# 라벨 저장
labels = kmeans.labels_

# 거리 계산시 사용하는 열 이름 따로 저장 (x들만 저장)
cols = dataset.columns

# 군집 필터링을 위해 열 생성
dataset['label'] = labels

/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


### 2. 군집 간 거리와 군집 내 거리를 구하고자 한다. 두 점의 거리는 '유클리디안 거리'로 계산한다.

군집 내 거리 종류는 세 가지가 있으며 각 정의는 아래와 같다  

- complete : 군집 내 모든 점들의 거리 중 최대값  
    - 군집에 n개의 점이 있으면 nC2 조합의 거리 중 최대 거리
- average : 군집 내 모든 점들의 거리 평균  
    - 군집에 n개의 점이 있으면 nC2 조합의 거리들의 평균 거리
- centroid : (군집의 중심과 군집 내 모든 점들과의 거리 평균) * 2   
    - 2를 곱하는 것에 주의

#### 2-1 군집별로 군집 내 complete 거리를 모두 구하고, 최대값 구해보기
#### 2-2 군집별로 군집 내 average 거리를 모두 구하고, 최대값을 구해보기
#### 2-3 군집별로 군집 내 centroid 거리를 모두 구하고, 최대값을 구해보기

In [7]:
from scipy.spatial.distance import euclidean, pdist, cdist

# 리스트 생성
intra_complete_dists = []
intra_average_dists = []
intra_centroid_dists = []

for label in set(labels):
    # 1. 군집 필터링 
    cluster = dataset[dataset['label'] == label][cols]
    
    # 2. 거리 구하기
    complete = pdist(cluster).max()
    average = pdist(cluster).mean()
    centroid = cdist(cluster, [cluster.mean()]).mean() * 2
    
    # 3. 거리 삽입
    intra_complete_dists.append(complete)
    intra_average_dists.append(average)
    intra_centroid_dists.append(centroid)
    
# A, B, C 구하기 (각 배열의 최대값)
A, B, C = map(lambda x: round(max(x), 2), [intra_complete_dists, intra_average_dists, intra_centroid_dists])

A, B, C

(4.62, 1.59, 2.25)

군집 간 거리 종류는 네 가지가 있으며 각 정의는 아래와 같다  
- single : 두 군집 간 가장 가까운 점들과의 거리  
- complete : 두 군집 간 가장 먼 점들과의 거리  
- average : 두 군집 간 모든 점들과의 거리 평균  
- centroid : 두 군집의 중심점끼리의 거리

#### 2-4 서로 다른 두 군집의 single 거리를 모두 구하고, 최소값 구해보기
#### 2-5 서로 다른 두 군집의 complete 거리를 모두 구하고, 최소값 구해보기
#### 2-6 서로 다른 두 군집의 average 거리를 모두 구하고, 최소값 구해보기
#### 2-7 서로 다른 두 군집의 centroid 거리를 모두 구하고, 최소값 구해보기

In [8]:
from itertools import combinations

# 리스트 생성
inter_single_dists = []
inter_complete_dists = []
inter_average_dists = []
inter_centroid_dists = []

for label1, label2 in combinations(set(labels), 2):
    # 1. 군집 필터링
    cluster1 = dataset[dataset['label'] == label1][cols]
    cluster2 = dataset[dataset['label'] == label2][cols]
    
    # 2. 거리 계산
    single = cdist(cluster1, cluster2).min()
    complete = cdist(cluster1, cluster2).max()
    average = cdist(cluster1, cluster2).mean()
    centroid = euclidean(cluster1.mean(), cluster2.mean())
    
    # 3. 거리 삽입
    inter_single_dists.append(single)
    inter_complete_dists.append(complete)
    inter_average_dists.append(average)
    inter_centroid_dists.append(centroid)

# D, E, F, G 구하기 (각 배열의 최소값)
D, E, F, G = map(lambda x: round(min(x), 2), \
                 [inter_single_dists, inter_complete_dists, inter_average_dists, inter_centroid_dists])

D, E, F, G

(4.91, 11.37, 8.32, 8.2)

### 3. (군집 간 최소 거리 / 군집 내 최대 거리) 를 Dunn Index라고 한다. 
#### A\~C 중 최대값과 D\~G 중 최소값으로 Dunn Index를 구해보자

In [9]:
H = round(min(D, E, F, G) / max(A, B, C), 2)

H

1.06

Silhouette 계수는 개별 데이터가 가지는 군집화 지표로, 식은 아래와 같다.

### 실루엣 계수 = (b_i - a_i) / max(a_i, b_i)

- a_i = mean intra-cluster distance (점 i가 속한 군집 내의 모든 점들과의 거리 평균)
  - hint : cdist(cluster, [data_point]).sum() / (len(cluster) - 1)
- b_i = mean nearest-cluster distance (점 i와 다른 군집 데이터들과의 평균 거리를 모두 구하고, 그 중 최소값)

### 4. 모든 점들의 실루엣 계수를 구하고, 최대값을 I, 최소값을 J 라 하자.

In [10]:
# 실루엣 계수 리스트
sil_samples = []
    
# 모든 점들 순차적 순회하면서 실루엣 계수 구하기
for data, label1 in zip(dataset[cols].values, dataset.label):
    # 1. data 군집 필터링
    cluster1 = dataset[dataset['label'] == label1][cols]
    
    # 2. a_i 구하기
    a_i = cdist(cluster1, [data]).sum() / (len(cluster1) - 1)
    
    # 3. data와 다른 군집의 거리 구하기 (b_i 후보)
    b_i_ary = []
    
    for label2 in set(labels):
        # 같은 군집이면 pass
        if label1 == label2:
            continue
            
        # b_i 계산
        cluster2 = dataset[dataset['label'] == label2][cols]
        b_i_cand = cdist(cluster2, [data]).mean()
        b_i_ary.append(b_i_cand)
        
    # b_i 중 최소값이 b_i임
    b_i = min(b_i_ary)
    
    # 실루엣 계수 계산 후 리스트에 삽입
    sil = (b_i - a_i) / max(a_i, b_i)
    sil_samples.append(sil)


I, J = map(lambda x: round(x, 2), [max(sil_samples), min(sil_samples)])
I, J

(0.9, 0.66)

### 5. 실루엣 계수의 평균값이 Silhouette Score이다. Silhouette Score 구해보자

In [11]:
K = round(np.mean(sil_samples), 2)
K

0.83

In [12]:
# 라이브러리 사용한 값과 비교
from sklearn.metrics import silhouette_samples

sam = silhouette_samples(dataset[cols], labels)

i, j, k = map(lambda x: round(x, 2), [max(sam), min(sam), np.mean(sam)])
i, j, k

(0.9, 0.66, 0.83)